This script will take the predictors and add a band called burn fraction from fire cci and save the new tif files. This will eventually be used to make a parquet file which is the training data frame

Make parquet file

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import re
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.windows import Window
from rasterio.warp import transform as rio_transform
from rasterio.features import rasterize
from pathlib import Path
from tqdm import tqdm

import geopandas as gpd

import pyarrow as pa
import pyarrow.parquet as pq

# ================== CONFIG ==================
IN_DIR   = "/explore/nobackup/people/spotter5/clelland_fire_ml/training_e5l_cems_firecci_with_fraction"

# One parquet file per ecoregion will be written here
OUT_ECO_DIR = "/explore/nobackup/people/spotter5/clelland_fire_ml/parquet_cems_by_ecoregion"
os.makedirs(OUT_ECO_DIR, exist_ok=True)

# Ecoregion shapefile (same as in your burned-area script)
ECOS_PATH  = "/explore/nobackup/people/spotter5/helene/raw/merge_eco_v2.shp"
ECO_ID_COL = "ecoregion"   # column in shapefile that identifies each ecoregion

# If your TIFF CRS is projected (e.g., EPSG:3413) but you want true lat/lon degrees:
REPROJECT_TO_EPSG4326 = True

# ================== HELPERS ==================

def sanitize_names(names):
    """Sanitize band names to safe column names and ensure uniqueness."""
    seen = {}
    out = []
    for n in names:
        if n is None or str(n).strip() == "":
            n = "band"
        n0 = re.sub(r"[^a-zA-Z0-9_]", "_", str(n).strip())
        n0 = re.sub(r"_+", "_", n0).strip("_")
        if n0 == "":
            n0 = "band"
        if n0 in seen:
            seen[n0] += 1
            n0 = f"{n0}_{seen[n0]}"
        else:
            seen[n0] = 1
        out.append(n0)
    return out

name_re = re.compile(r"cems_e5l_firecci_(\d{4})_(\d{1,2})", re.IGNORECASE)
def parse_year_month(fname: str):
    m = name_re.search(fname)
    if not m:
        return None, None
    return int(m.group(1)), int(m.group(2))

def windows_for_dataset(ds: rio.DatasetReader):
    """Iterate using native block windows when possible (COG-friendly)."""
    try:
        for _, w in ds.block_windows(1):
            yield w
    except Exception:
        tile = 1024
        for row_off in range(0, ds.height, tile):
            height = min(tile, ds.height - row_off)
            for col_off in range(0, ds.width, tile):
                width = min(tile, ds.width - col_off)
                yield Window(col_off, row_off, width, height)

def sanitize_ecoregion_filename(eco_label) -> str:
    """Turn an ecoregion label into a safe filename."""
    s = str(eco_label)
    s = re.sub(r"[^a-zA-Z0-9_-]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    if not s:
        s = "unknown_ecoregion"
    return s

# ================== COLLECT TIFS ==================

print("Collecting TIFFs...")
tifs = sorted(Path(IN_DIR).glob("cems_e5l_firecci_*with_fraction.tif"))
if not tifs:
    tifs = sorted(Path(IN_DIR).glob("cems_e5l_firecci_*.tif"))

if not tifs:
    raise FileNotFoundError(f"No CEMS TIFFs found in {IN_DIR}")

print(f"Found {len(tifs)} TIFFs. Using the first as template for ecoregion raster.")
template_tif = tifs[0]

# ================== BUILD ECOREGION RASTER (INTERNAL MASK ONLY) ==================

print("Loading ecoregions shapefile...")
ecos = gpd.read_file(ECOS_PATH)
if ECO_ID_COL not in ecos.columns:
    raise ValueError(
        f"ECO_ID_COL='{ECO_ID_COL}' not found in {ECOS_PATH}. "
        f"Columns: {list(ecos.columns)}"
    )

ecos = ecos[ecos.geometry.notnull()].copy()
if ecos.empty:
    raise ValueError("Ecoregion GeoDataFrame has no valid geometries.")

with rio.open(template_tif) as tmpl_ds:
    tmpl_crs = tmpl_ds.crs
    tmpl_transform = tmpl_ds.transform
    tmpl_height = tmpl_ds.height
    tmpl_width = tmpl_ds.width

    if tmpl_crs is None:
        raise ValueError(f"Template {template_tif} has no CRS; cannot rasterize ecoregions.")

    print(f"Reprojecting ecoregions to raster CRS: {tmpl_crs}")
    ecos = ecos.to_crs(tmpl_crs)

    # Assign integer codes for each ecoregion row (internal)
    ecos["eco_code"] = np.arange(1, len(ecos) + 1, dtype=np.int32)
    # Map code -> label (string)
    code_to_label = dict(zip(ecos["eco_code"], ecos[ECO_ID_COL]))

    shapes = [(geom, code) for geom, code in zip(ecos.geometry, ecos["eco_code"])]

    print("Rasterizing ecoregions to template grid...")
    eco_raster = rasterize(
        shapes,
        out_shape=(tmpl_height, tmpl_width),
        transform=tmpl_transform,
        fill=0,            # 0 = no ecoregion
        dtype="int32",
    )

print("Ecoregion raster built successfully.")

# Save mapping of eco_code -> ecoregion name (for reference)
mapping_csv = os.path.join(OUT_ECO_DIR, "ecoregion_code_mapping.csv")
pd.DataFrame(
    {
        "eco_code": list(code_to_label.keys()),
        "ecoregion": list(code_to_label.values())
    }
).to_csv(mapping_csv, index=False)
print(f"Saved code->ecoregion mapping to: {mapping_csv}")

# ================== PARQUET WRITERS (ONE PER ECOREGION) ==================

# Dictionary: eco_code -> (ParquetWriter, filepath)
writers = {}

def get_writer_for_ecoregion(eco_code: int, schema: pa.Schema, eco_label: str):
    """
    Lazily create (and cache) a ParquetWriter for each ecoregion.
    We *do not* store eco_code in the parquet, only the data + lon/lat + year/month.
    """
    if eco_code in writers:
        return writers[eco_code]["writer"]

    fn_safe = sanitize_ecoregion_filename(eco_label)
    outfile = os.path.join(OUT_ECO_DIR, f"ecoregion_{fn_safe}.parquet")

    writer = pq.ParquetWriter(outfile, schema)
    writers[eco_code] = {"path": outfile, "writer": writer}
    return writer

# ================== MAIN LOOP ==================

# Optional: track a canonical column order from the first file for consistency
canonical_cols = None

template_crs = tmpl_crs
template_transform = tmpl_transform
template_height = tmpl_height
template_width = tmpl_width

for tif in tqdm(tifs, desc="Building per-ecoregion Parquet files"):
    year, month = parse_year_month(tif.name)
    if year is None:
        print(f"[SKIP] {tif.name} (no YYYY_M in name)")
        continue

    with rio.open(tif) as ds:
        # Ensure this TIFF matches the template grid
        if (
            ds.crs != template_crs
            or ds.transform != template_transform
            or ds.height != template_height
            or ds.width != template_width
        ):
            raise ValueError(
                f"Grid mismatch for {tif}:\n"
                f"  CRS: {ds.crs} vs {template_crs}\n"
                f"  Transform: {ds.transform} vs {template_transform}\n"
                f"  Shape: ({ds.height}, {ds.width}) vs ({template_height}, {template_width})"
            )

        # Band names
        band_names = list(ds.descriptions)
        if not any(band_names):
            band_names = [f"B{i}" for i in range(1, ds.count + 1)]
        safe_names = sanitize_names(band_names)

        # Establish canonical column order once (bands first)
        if canonical_cols is None:
            canonical_cols = safe_names + ["longitude", "latitude", "year", "month"]

        # Process in windows
        for w in windows_for_dataset(ds):
            data = ds.read(indexes=list(range(1, ds.count + 1)), window=w).astype(np.float32)  # (bands, h, w)
            h, w_cols = data.shape[1], data.shape[2]

            if h == 0 or w_cols == 0:
                continue

            # Row/col grid for this window
            rows = np.arange(int(w.row_off), int(w.row_off + h))
            cols = np.arange(int(w.col_off), int(w.col_off + w_cols))
            rr, cc = np.meshgrid(rows, cols, indexing="ij")

            # Pixel center coords in native CRS
            xs, ys = rio.transform.xy(ds.transform, rr, cc, offset="center")
            xs = np.asarray(xs, dtype=np.float64).reshape(-1)
            ys = np.asarray(ys, dtype=np.float64).reshape(-1)

            # Reproject to lon/lat if desired
            if (
                REPROJECT_TO_EPSG4326
                and ds.crs is not None
                and ds.crs.to_string().upper() not in ("EPSG:4326", "OGC:CRS84")
            ):
                lons, lats = rio_transform(ds.crs, "EPSG:4326", xs, ys)
                lons = np.array(lons, dtype=np.float64)
                lats = np.array(lats, dtype=np.float64)
            else:
                lons, lats = xs, ys

            # Flatten bands to rows
            arr2d = data.reshape(ds.count, -1).T  # (h*w, bands)
            df = pd.DataFrame(arr2d, columns=safe_names)

            # Add coords + partitions
            df["longitude"] = lons
            df["latitude"]  = lats
            df["year"]      = year
            df["month"]     = month

            # Slice ecoregion raster for this window and flatten (internal only)
            eco_window = eco_raster[
                int(w.row_off): int(w.row_off + h),
                int(w.col_off): int(w.col_off + w_cols)
            ].reshape(-1)

            # Attach eco_code temporarily for grouping
            df["eco_code"] = eco_window.astype(np.int32)

            # Drop pixels outside all ecoregions
            df = df[df["eco_code"] != 0]
            if df.empty:
                continue

            # Align column order to canonical set
            for col in canonical_cols:
                if col not in df.columns:
                    df[col] = np.nan
            # Note: don't include eco_code in canonical_cols
            df = df[canonical_cols + ["eco_code"]]

            # Group by eco_code and write to that ecoregion's Parquet file
            for eco_code, df_grp in df.groupby("eco_code"):
                eco_label = code_to_label.get(eco_code, f"eco_{eco_code}")

                # Drop eco_code from output (you said you don't need it per pixel)
                df_out = df_grp.drop(columns=["eco_code"])

                # Convert to Arrow table
                table = pa.Table.from_pandas(df_out, preserve_index=False)

                # Create writer lazily with this schema
                writer = get_writer_for_ecoregion(eco_code, table.schema, eco_label)
                writer.write_table(table)

# Close all Parquet writers
for eco_code, winfo in writers.items():
    if winfo["writer"] is not None:
        winfo["writer"].close()

print("\n✅ Done.")
print(f"Per-ecoregion Parquet files written to:\n  {OUT_ECO_DIR}")
print("Each file contains only pixels within that ecoregion (bands + lon/lat + year + month).")


Found 228 TIFFs. Using the first as template for ecoregion raster.
Loading ecoregions shapefile...
Reprojecting ecoregions to raster CRS: EPSG:6931
Rasterizing ecoregions to template grid...
Ecoregion raster built successfully.
Saved code->ecoregion mapping to: /explore/nobackup/people/spotter5/clelland_fire_ml/parquet_cems_by_ecoregion/ecoregion_code_mapping.csv


Building per-ecoregion Parquet files:  99%|█████████▊| 225/228 [44:30<00:49, 16.63s/it] 

Save to geotiff per ecoregion

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import re
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.windows import Window
from rasterio.features import rasterize
from pathlib import Path
from tqdm import tqdm

import geopandas as gpd

# ================== CONFIG ==================
IN_DIR   = "/explore/nobackup/people/spotter5/clelland_fire_ml/training_e5l_cems_firecci_with_fraction"

# Root directory: one subfolder per ecoregion, TIFFs inside
OUT_ECO_TIF_DIR = "/explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion"
os.makedirs(OUT_ECO_TIF_DIR, exist_ok=True)

# Ecoregion shapefile (same as in your burned-area script)
ECOS_PATH  = "/explore/nobackup/people/spotter5/helene/raw/merge_eco_v2.shp"
ECO_ID_COL = "ecoregion"   # column in shapefile that identifies each ecoregion

# If your TIFF CRS is projected (e.g., EPSG:3413) but you want true lat/lon degrees:
# (Not needed for the GeoTIFF outputs themselves; we keep native grid.)
REPROJECT_TO_EPSG4326 = False  # not used now, but left for clarity

# Preferred substring in band description for the "fraction" band
FRACTION_BAND_NAME_SUBSTR = "fraction"

# NODATA value for output rasters (uint8), different from 0/1
OUT_NODATA = 255

# ================== HELPERS ==================

def sanitize_names(names):
    """Sanitize band names to safe column names and ensure uniqueness."""
    seen = {}
    out = []
    for n in names:
        if n is None or str(n).strip() == "":
            n = "band"
        n0 = re.sub(r"[^a-zA-Z0-9_]", "_", str(n).strip())
        n0 = re.sub(r"_+", "_", n0).strip("_")
        if n0 == "":
            n0 = "band"
        if n0 in seen:
            seen[n0] += 1
            n0 = f"{n0}_{seen[n0]}"
        else:
            seen[n0] = 1
        out.append(n0)
    return out

name_re = re.compile(r"cems_e5l_firecci_(\d{4})_(\d{1,2})", re.IGNORECASE)
def parse_year_month(fname: str):
    m = name_re.search(fname)
    if not m:
        return None, None
    return int(m.group(1)), int(m.group(2))

def windows_for_dataset(ds: rio.DatasetReader):
    """Iterate using native block windows when possible (COG-friendly)."""
    try:
        for _, w in ds.block_windows(1):
            yield w
    except Exception:
        tile = 1024
        for row_off in range(0, ds.height, tile):
            height = min(tile, ds.height - row_off)
            for col_off in range(0, ds.width, tile):
                width = min(tile, ds.width - col_off)
                yield Window(col_off, row_off, width, height)

def sanitize_ecoregion_filename(eco_label) -> str:
    """Turn an ecoregion label into a safe folder name."""
    s = str(eco_label)
    s = re.sub(r"[^a-zA-Z0-9_-]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    if not s:
        s = "unknown_ecoregion"
    return s

def get_fraction_band_index(ds: rio.DatasetReader, substr: str = FRACTION_BAND_NAME_SUBSTR) -> int:
    """
    Try to find the band index whose description contains 'substr' (case-insensitive).
    If not found, fall back to the last band.
    Returns a 1-based band index (rasterio convention).
    """
    descs = list(ds.descriptions) if ds.descriptions is not None else []
    for i, d in enumerate(descs, start=1):
        if d and substr.lower() in d.lower():
            print(f"  Using band {i} ('{d}') as fraction band.")
            return i
    # fallback: last band
    print(f"  No band descriptions matching '{substr}' found; using last band ({ds.count}).")
    return ds.count

# ================== COLLECT TIFS ==================

print("Collecting TIFFs...")
tifs = sorted(Path(IN_DIR).glob("cems_e5l_firecci_*with_fraction.tif"))
if not tifs:
    tifs = sorted(Path(IN_DIR).glob("cems_e5l_firecci_*.tif"))

if not tifs:
    raise FileNotFoundError(f"No CEMS TIFFs found in {IN_DIR}")

print(f"Found {len(tifs)} TIFFs. Using the first as template for ecoregion raster.")
template_tif = tifs[0]

# ================== BUILD ECOREGION RASTER (INTERNAL MASK ONLY) ==================

print("Loading ecoregions shapefile...")
ecos = gpd.read_file(ECOS_PATH)
if ECO_ID_COL not in ecos.columns:
    raise ValueError(
        f"ECO_ID_COL='{ECO_ID_COL}' not found in {ECOS_PATH}. "
        f"Columns: {list(ecos.columns)}"
    )

ecos = ecos[ecos.geometry.notnull()].copy()
if ecos.empty:
    raise ValueError("Ecoregion GeoDataFrame has no valid geometries.")

with rio.open(template_tif) as tmpl_ds:
    tmpl_crs = tmpl_ds.crs
    tmpl_transform = tmpl_ds.transform
    tmpl_height = tmpl_ds.height
    tmpl_width = tmpl_ds.width

    if tmpl_crs is None:
        raise ValueError(f"Template {template_tif} has no CRS; cannot rasterize ecoregions.")

    print(f"Reprojecting ecoregions to raster CRS: {tmpl_crs}")
    ecos = ecos.to_crs(tmpl_crs)

    # Assign integer codes for each ecoregion row (internal)
    ecos["eco_code"] = np.arange(1, len(ecos) + 1, dtype=np.int32)
    # Map code -> label (string)
    code_to_label = dict(zip(ecos["eco_code"], ecos[ECO_ID_COL]))

    shapes = [(geom, code) for geom, code in zip(ecos.geometry, ecos["eco_code"])]

    print("Rasterizing ecoregions to template grid...")
    eco_raster = rasterize(
        shapes,
        out_shape=(tmpl_height, tmpl_width),
        transform=tmpl_transform,
        fill=0,            # 0 = no ecoregion
        dtype="int32",
    )

print("Ecoregion raster built successfully.")

# Save mapping of eco_code -> ecoregion name (for reference)
mapping_csv = os.path.join(OUT_ECO_TIF_DIR, "ecoregion_code_mapping.csv")
pd.DataFrame(
    {
        "eco_code": list(code_to_label.keys()),
        "ecoregion": list(code_to_label.values())
    }
).to_csv(mapping_csv, index=False)
print(f"Saved code->ecoregion mapping to: {mapping_csv}")

# Unique eco codes to process (exclude 0)
eco_codes = [c for c in np.unique(eco_raster) if c != 0]
print(f"Will export GeoTIFFs for {len(eco_codes)} ecoregions.")

template_crs = tmpl_crs
template_transform = tmpl_transform
template_height = tmpl_height
template_width = tmpl_width

# ================== MAIN LOOP: WRITE GEOTIFFS ==================

for tif in tqdm(tifs, desc="Writing per-ecoregion GeoTIFFs"):
    year, month = parse_year_month(tif.name)
    if year is None:
        print(f"[SKIP] {tif.name} (no YYYY_M in name)")
        continue

    with rio.open(tif) as ds:
        # Ensure this TIFF matches the template grid
        if (
            ds.crs != template_crs
            or ds.transform != template_transform
            or ds.height != template_height
            or ds.width != template_width
        ):
            raise ValueError(
                f"Grid mismatch for {tif}:\n"
                f"  CRS: {ds.crs} vs {template_crs}\n"
                f"  Transform: {ds.transform} vs {template_transform}\n"
                f"  Shape: ({ds.height}, {ds.width}) vs ({template_height}, {template_width})"
            )

        print(f"\nProcessing {tif.name} (year={year}, month={month})")

        frac_band_idx = get_fraction_band_index(ds)

        # Prepare output datasets for each ecoregion for this year/month
        eco_dst = {}
        base_profile = ds.profile.copy()
        base_profile.update(
            driver="GTiff",
            count=1,
            dtype="uint8",
            compress="LZW",
            nodata=OUT_NODATA
        )

        for eco_code in eco_codes:
            eco_label = code_to_label.get(eco_code, f"eco_{eco_code}")
            eco_folder = os.path.join(
                OUT_ECO_TIF_DIR,
                sanitize_ecoregion_filename(eco_label)
            )
            os.makedirs(eco_folder, exist_ok=True)

            out_name = f"{year}_{month:02d}_binary.tif"
            out_path = os.path.join(eco_folder, out_name)

            dst = rio.open(out_path, "w", **base_profile)
            eco_dst[eco_code] = dst

        # Stream over windows: threshold + write to each ecoregion raster
        for w in windows_for_dataset(ds):
            h = int(w.height)
            w_cols = int(w.width)
            if h == 0 or w_cols == 0:
                continue

            # Read fraction band for this window
            frac_window = ds.read(frac_band_idx, window=w).astype(np.float32)  # shape (h, w)

            # Slice ecoregion raster for this window
            eco_window = eco_raster[
                int(w.row_off): int(w.row_off + h),
                int(w.col_off): int(w.col_off + w_cols)
            ]

            if eco_window.size == 0:
                continue

            # Binary classification: > 0.5 -> 1, <= 0.5 -> 0
            # NaNs become 0 by default (comparison False).
            binary_full = (frac_window > 0.5).astype(np.uint8)

            # For each ecoregion present in this window, write only those pixels
            present_codes = np.unique(eco_window)
            present_codes = present_codes[present_codes != 0]  # exclude outside any eco

            for eco_code in present_codes:
                dst = eco_dst.get(eco_code)
                if dst is None:
                    continue

                mask = (eco_window == eco_code)
                if not mask.any():
                    continue

                # Start with NODATA everywhere, then fill where mask is True
                out_block = np.full(mask.shape, OUT_NODATA, dtype=np.uint8)
                out_block[mask] = binary_full[mask]

                dst.write(out_block, 1, window=w)

        # Close all output rasters for this year/month
        for eco_code, dst in eco_dst.items():
            dst.close()
            eco_label = code_to_label.get(eco_code, f"eco_{eco_code}")
            eco_folder = os.path.join(
                OUT_ECO_TIF_DIR,
                sanitize_ecoregion_filename(eco_label)
            )
            out_name = f"{year}_{month:02d}_binary.tif"
            print(f"  Wrote {os.path.join(eco_folder, out_name)}")

print("\n✅ Done.")
print(f"Per-ecoregion GeoTIFFs written under:\n  {OUT_ECO_TIF_DIR}")
print("Each ecoregion folder contains one uint8 LZW-compressed binary raster per year/month.")


Found 228 TIFFs. Using the first as template for ecoregion raster.
Loading ecoregions shapefile...
Reprojecting ecoregions to raster CRS: EPSG:6931
Rasterizing ecoregions to template grid...
Ecoregion raster built successfully.
Saved code->ecoregion mapping to: /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ecoregion_code_mapping.csv
Will export GeoTIFFs for 23 ecoregions.


Writing per-ecoregion GeoTIFFs:   0%|          | 0/228 [00:00<?, ?it/s]


Processing cems_e5l_firecci_2001_10_with_fraction.tif (year=2001, month=10)
  Using band 16 ('fraction') as fraction band.


Writing per-ecoregion GeoTIFFs:   0%|          | 1/228 [00:01<06:43,  1.78s/it]

  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_BOREAL_INTERIOR/2001_10_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_TUNDRA/2001_10_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ARCTIC_CORDILLERA/2001_10_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Arctic_Deserts_and_Tundra/2001_10_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_CORDILLERA/2001_10_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_PLAIN/2001_10_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BROOKS_RANGE_TUNDRA/2001_10_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Central_Taiga/2001_10_binary.tif
  Wrote /explore/nobac

...

  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_BOREAL_INTERIOR/2019_07_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_TUNDRA/2019_07_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ARCTIC_CORDILLERA/2019_07_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Arctic_Deserts_and_Tundra/2019_07_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_CORDILLERA/2019_07_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_PLAIN/2019_07_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BROOKS_RANGE_TUNDRA/2019_07_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Central_Taiga/2019_07_binary.tif
  Wrote /explore/nobac

Writing per-ecoregion GeoTIFFs: 100%|█████████▉| 227/228 [13:20<00:05,  5.59s/it]

  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_BOREAL_INTERIOR/2019_08_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_TUNDRA/2019_08_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ARCTIC_CORDILLERA/2019_08_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Arctic_Deserts_and_Tundra/2019_08_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_CORDILLERA/2019_08_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_PLAIN/2019_08_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BROOKS_RANGE_TUNDRA/2019_08_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Central_Taiga/2019_08_binary.tif
  Wrote /explore/nobac

Writing per-ecoregion GeoTIFFs: 100%|██████████| 228/228 [13:26<00:00,  3.54s/it]

  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_BOREAL_INTERIOR/2019_09_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ALASKA_TUNDRA/2019_09_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/ARCTIC_CORDILLERA/2019_09_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Arctic_Deserts_and_Tundra/2019_09_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_CORDILLERA/2019_09_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BOREAL_PLAIN/2019_09_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/BROOKS_RANGE_TUNDRA/2019_09_binary.tif
  Wrote /explore/nobackup/people/spotter5/clelland_fire_ml/geotiff_cems_by_ecoregion/Central_Taiga/2019_09_binary.tif
  Wrote /explore/nobac

In [3]:
't'

't'